In [1]:
from models.common import DetectMultiBackend
import torch
import matplotlib.pyplot as plt
from utils.augmentations import letterbox
from utils.dataloaders import IMG_FORMATS, VID_FORMATS, LoadImages
from utils.plots import Annotator, colors, save_one_box
from utils.general import (LOGGER, check_file, check_img_size, check_imshow, check_requirements, colorstr, cv2,
                           increment_path, non_max_suppression, print_args, scale_coords, strip_optimizer, xyxy2xywh)
import numpy as np
import os

/home/vishwesh/anaconda3/envs/clsenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%matplotlib inline

In [4]:
WEIGHTS_PATH = r"best.pt"
DEVICE = "cpu"
half = False
model = DetectMultiBackend(WEIGHTS_PATH, device=DEVICE, dnn=False)

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 7249215 parameters, 0 gradients, 16.7 GFLOPs


In [5]:
# model

In [6]:
stride, names, pt = model.stride, model.names, model.pt

In [7]:
dataset = LoadImages("images/", img_size=(640, 640), stride=stride, auto=pt)

['/home/vishwesh/Desktop/study/sem7/prac/DL/PROJ/images/IMG-20220311-WA0001.jpg', '/home/vishwesh/Desktop/study/sem7/prac/DL/PROJ/images/Snapchat-1086665998.jpg', '/home/vishwesh/Desktop/study/sem7/prac/DL/PROJ/images/Snapchat-1272617603.jpg', '/home/vishwesh/Desktop/study/sem7/prac/DL/PROJ/images/Snapchat-1915067342.jpg', '/home/vishwesh/Desktop/study/sem7/prac/DL/PROJ/images/Snapchat-607935325.jpg']


In [8]:
dt = [0.0,0.0, 0.0]

In [14]:
conf_thres=0.25  # confidence threshold
iou_thres=0.45
classes=None
save_crop = False
line_thickness = 3

In [10]:
real_classes = ["with helmet","without helmet"]

In [11]:
def transform(point, real, new):
    return int(point * new/real)

In [15]:
RESIZE = (720, 560)
COLOR_BORDER = [0, 0, 255]
COLOR_TEXT = [0, 255, 0]
save_path = "/home/vishwesh/Desktop"
img_path = "/home/vishwesh/Desktop/study/sem7/Project/content/yolov5/data/images/one.jpg"
file_name = img_path.split("/")[-1].split(".")[0]
img0 = cv2.imread(img_path)
img_size = 640
stride = 32
auto = True
img = letterbox(img0, img_size, stride=stride, auto=auto)[0]
img = img.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
img = np.ascontiguousarray(img)
im = torch.from_numpy(img).to(DEVICE)
im = im.float()
im = im/255
im = im[None]
pred = model(im, augment=False, visualize=False)
    # t3 = time_sync()
    # dt[1] += t3 - t2
# print("hello")


# print(pred.shape)
# NMS
pred = non_max_suppression(pred, conf_thres, iou_thres, classes, False, max_det=1000)
# print(pred)
for i, det in enumerate(pred):
    p, im0 = img_path, img0.copy()
    gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
    imc = im0.copy() if save_crop else im0  # for save_crop
    real_width, real_height, _ = im0.shape
    new = cv2.resize(im0, (RESIZE[0], RESIZE[1]))
    print(new.shape)
    if len(det):
        det[:, :4] = scale_coords(im.shape[2:], det[:, :4], img0.shape).round()
        # print(det)
        # new = cv2.rectangle(img0, (x_start, y_start), (x_end, y_end), COLOR, 5)
        for j in det:
            # print(j)
            # x_start, y_start, x_end, y_end = int(j[0]),int(j[1]),int(j[2]),int(j[3])
            x_start, y_start, x_end, y_end = (
                transform(j[0], real_height, RESIZE[0]),
                transform(j[1], real_width, RESIZE[1]),
                transform(j[2], real_height, RESIZE[0]),
                transform(j[3], real_width, RESIZE[1])
            )
            class_idx = int(j[-1])
            conf = j[-2]
            new = cv2.rectangle(new, (x_start, y_start), (x_end, y_end), COLOR_BORDER, 2)
            # cv2.putText()
            new = cv2.putText(new, real_classes[int(class_idx)]+" "+str(np.round(conf.numpy(),2)),(x_start, y_start-10),1,1.5,COLOR_TEXT,2)
            # print(real_width)
            # print(real_height)
            # print((int(x_start*480/real_width)+20, int(y_start*480/real_height)-10))
            if y_start <50:
                new = cv2.putText(new, real_classes[int(class_idx)]+" "+str(np.round(conf.numpy(),2)),(x_start, y_end+10),1,1.5,COLOR_TEXT,2)
            else : 
                new = cv2.putText(new, real_classes[int(class_idx)]+" "+str(np.round(conf.numpy(),2)),(x_start, y_start-10),1,1.5,COLOR_TEXT,2)
            

             
            # print((int(x_start*720/real_width), int(y_start*720/real_height)-5))
            # print(real_classes[class_idx])
            # new = cv2.putText(new, real_classes[int(class_idx)]+" "+str(conf.numpy()),(int(x_start*720/real_height), int(y_start*720/real_width)-5),1,0.8,COLOR,1)
            # new = cv2.putText(new, real_classes[class_idx],(x_start, y_start-10),1,1.1,[255, 255, 255],1)
            
            # cv2.imwrite(os.path.join(save_path,"detected_{}.jpeg".format(file_name)),new)
    cv2.imshow("Frame", new)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite(os.path.join(save_path,"detected_{}.jpeg".format(file_name)),new)

(560, 720, 3)


In [112]:
for path, im, im0s, vid_cap, s in dataset:
    # t1 = time_sync()
    # plt.imshow(im0s)
    # plt.show()
    im = torch.from_numpy(im).to(DEVICE)
    print(im)
    # im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
    im = im.float()
    im /= 255  # 0 - 255 to 0.0 - 1.0
    if len(im.shape) == 3:
        im = im[None]  # expand for batch dim
    # t2 = time_sync()
    # dt[0] += t2 - t1

    # Inference
    # visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visualize else False
    pred = model(im, augment=False, visualize=False)
    # t3 = time_sync()
    # dt[1] += t3 - t2
    print(pred.shape)
    # NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres, classes, False, max_det=1000)
    for i, det in enumerate(pred):
        print(im0.shape)
        p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)
        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
        imc = im0.copy() if save_crop else im0  # for save_crop
        annotator = Annotator(im0, line_width=line_thickness, example=str(names))
        if len(det):
            det[:, :4] = scale_coords(im.shape[2:], det[:, :4], im0.shape).round()
            # print(det)
            for j in det:
                print(j)
                x_start, y_start, x_end, y_end = int(j[0]),int(j[1]),int(j[2]),int(j[3])
                class_idx = int(j[-1])
                conf = j[-2]
                new = cv2.rectangle(im0s, (x_start, y_start), (x_end, y_end), [0, 255, 0], 5)
                real_width, real_height, _ = im0s.shape
                print(real_width)
                print(real_height)
                # print((int(x_start*480/real_width)+20, int(y_start*480/real_height)-10))
                new = cv2.resize(new, (720, 720))
                print((int(x_start*720/real_width), int(y_start*720/real_height)-5))
                print(real_classes[class_idx])
                new = cv2.putText(new, real_classes[int(class_idx)]+" "+str(conf.numpy()),(int(x_start*720/real_height), int(y_start*720/real_width)-10),1,0.8,[0, 255, 0],1)
                # new = cv2.putText(new, real_classes[class_idx],(x_start, y_start-10),1,1.1,[255, 255, 255],1)
                cv2.imshow("Frame", new)
                
                cv2.waitKey(0)
                cv2.destroyAllWindows()
    # break


        

tensor([[[114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         ...,
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114]],

        [[114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         ...,
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114]],

        [[114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         ...,
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114],
         [114, 114, 114,  ..., 114, 114, 114]]], dtype=torch.uint8)
torch.Size([1, 13860, 7])
(2340, 1080, 3)
tensor([250.00000,  21.00000, 427.00000, 254.

In [58]:
250 * 480/1280

93.75